<a href="https://colab.research.google.com/github/adumont/ml-dl-notes/blob/master/PyTorch/pytorch01_tensor_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline


What is PyTorch?
================

It’s a Python-based scientific computing package targeted at two sets of
audiences:

-  A replacement for NumPy to use the power of GPUs
-  a deep learning research platform that provides maximum flexibility
   and speed

Getting Started
---------------

Tensors
^^^^^^^

Tensors are similar to NumPy’s ndarrays, with the addition being that
Tensors can also be used on a GPU to accelerate computing.



In [0]:
from __future__ import print_function
import torch

Construct a 5x3 matrix, uninitialized:



In [59]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_properties(0))
  device=torch.device('cuda')
else:
  device=torch.device('cpu')
print('device:',device)


_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)
device: cuda


In [34]:
x = torch.empty(5, 3).to(device)
print(x)

tensor([[-5.9602e-21,  0.0000e+00,  4.4842e-44],
        [ 0.0000e+00,         nan,  2.2726e-01],
        [ 1.3297e+22,  5.4411e-05,  4.2487e-05],
        [ 2.7326e-06,  2.1887e+23,  1.3167e-08],
        [ 2.1769e+23,  5.4719e+22,  1.5000e-01]], device='cuda:0')


Construct a randomly initialized matrix:



In [35]:
x = torch.rand(5, 3).to(device)
print(x)

tensor([[0.1496, 0.4119, 0.0138],
        [0.4405, 0.7533, 0.4896],
        [0.6399, 0.8583, 0.3098],
        [0.6392, 0.5771, 0.4444],
        [0.2094, 0.8070, 0.2140]], device='cuda:0')


Construct a matrix filled zeros and of dtype long:



In [36]:
x = torch.zeros(5, 3, dtype=torch.long).to(device)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], device='cuda:0')


Construct a tensor directly from data:



In [37]:
x = torch.tensor([5.5, 3]).to(device)
print(x)

tensor([5.5000, 3.0000], device='cuda:0')


or create a tensor based on an existing tensor. These methods
will reuse properties of the input tensor, e.g. dtype, unless
new values are provided by user



In [38]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='cuda:0', dtype=torch.float64)
tensor([[ 0.2757,  0.3263, -0.2283],
        [-1.0571,  0.1755, -0.1589],
        [-0.9766,  0.9888,  0.2861],
        [-1.6927, -0.7160,  0.2386],
        [-1.9513,  0.5927, -0.9816]], device='cuda:0')


Get its rank and dimensions:



In [41]:
print("Rank:", x.dim())
print("Dimensions:", x.size())

Rank: 2
Dimensions: torch.Size([5, 3])


<div class="alert alert-info"><h4>Note</h4><p>``torch.Size`` is in fact a tuple, so it supports all tuple operations.</p></div>

Operations
^^^^^^^^^^
There are multiple syntaxes for operations. In the following
example, we will take a look at the addition operation.

Addition: syntax 1



In [43]:
y = torch.rand(5, 3).to(device)
print(x + y)

tensor([[ 1.2663,  0.6400,  0.5192],
        [-0.2234,  0.9526,  0.2436],
        [-0.0486,  1.3933,  0.6793],
        [-1.6243,  0.1532,  1.0604],
        [-1.6374,  0.9684, -0.5788]], device='cuda:0')


Addition: syntax 2



In [44]:
print(torch.add(x, y))

tensor([[ 1.2663,  0.6400,  0.5192],
        [-0.2234,  0.9526,  0.2436],
        [-0.0486,  1.3933,  0.6793],
        [-1.6243,  0.1532,  1.0604],
        [-1.6374,  0.9684, -0.5788]], device='cuda:0')


Addition: providing an output tensor as argument



In [0]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

Addition: in-place



In [0]:
# adds x to y
y.add_(x)
print(y)

<div class="alert alert-info"><h4>Note</h4><p>Any operation that mutates a tensor in-place is post-fixed with an ``_``.
    For example: ``x.copy_(y)``, ``x.t_()``, will change ``x``.</p></div>

You can use standard NumPy-like indexing with all bells and whistles!



In [50]:
print(x[:, 2])

tensor([-0.2283, -0.1589,  0.2861,  0.2386, -0.9816], device='cuda:0')


In [46]:
print(x)

tensor([[ 0.2757,  0.3263, -0.2283],
        [-1.0571,  0.1755, -0.1589],
        [-0.9766,  0.9888,  0.2861],
        [-1.6927, -0.7160,  0.2386],
        [-1.9513,  0.5927, -0.9816]], device='cuda:0')


Resizing: If you want to resize/reshape tensor, you can use ``torch.view``:



In [53]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
t = x.view(2, -1)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size(), t.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8]) torch.Size([2, 8])


If you have a one element tensor, use ``.item()`` to get the value as a
Python number



In [54]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([1.6718])
1.6718454360961914


**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,
  mathematical operations, linear algebra, random numbers, etc.,
  are described
  `here <http://pytorch.org/docs/torch>`_.

NumPy Bridge
------------

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory
locations, and changing one will change the other.

Converting a Torch Tensor to a NumPy Array
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



In [55]:
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [56]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


See how the numpy array changed in value.



In [57]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


Converting NumPy Array to Torch Tensor
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
See how changing the np array changed the Torch Tensor automatically



In [58]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


All the Tensors on the CPU except a CharTensor support converting to
NumPy and back.

CUDA Tensors
------------

Tensors can be moved onto any device using the ``.to`` method.



In [60]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([2.6718], device='cuda:0')
tensor([2.6718], dtype=torch.float64)
